In [2]:
!/anaconda/envs/azureml_py310_sdkv2/bin/python -m pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /anaconda/envs/azureml_py310_sdkv2/bin/python -m pip install --upgrade pip


In [3]:
import pandas as pd

# Read your Excel file
df = pd.read_excel('got_persona_dataset_100.xlsx')

# Clean data
df = df.dropna(how='all', axis=0).dropna(how='all', axis=1)
df.columns = df.columns.str.strip().str.replace('[^A-Za-z0-9_]', '_', regex=True)

# Save as CSV
df.to_csv('got_persona_dataset_100.csv', index=False, encoding='utf-8')
print(f"✓ Converted! Shape: {df.shape}")

✓ Converted! Shape: (100, 21)


In [4]:
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()
datastore = ws.get_default_datastore()

# Upload and register
datastore.upload_files(['got_persona_dataset_100.csv'], target_path='datasets/', overwrite=True)
from azureml.data.datapath import DataPath
dataset = Dataset.Tabular.from_delimited_files(DataPath(datastore, 'datasets/got_persona_dataset_100.csv'))
dataset.register(ws, 'got-persona-dataset', create_new_version=True)

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/__init__.py:41: UserWarning: Versions of mlflow (3.1.1) and mlflow-skinny (2.22.1) are different. This may lead to unexpected behavior. Please install the same version of both packages.
  mlflow.mismatch._check_version_mismatch()
Message: [NOT_SUPPORTED_API_USE_ATTEMPT] The [_get_steps] API has been deprecated and is no longer supported
Payload: {"pid": 5511, "rslex_version": "2.22.5", "api_name": "_get_steps", "version": "5.1.6"}


Uploading an estimated of 1 files
Uploading got_persona_dataset_100.csv
Uploaded got_persona_dataset_100.csv, 1 files out of an estimated total of 1
Uploaded 1 files


{
  "definition": "EnginelessDataflow:\n---\ntype: mltable\npaths:\n  - pattern: \"azureml://subscriptions/ebc57341-e881-4473-9659-d8076026bd45/resourcegroups/azure-ai/workspaces/demeyer-robin-ml/datastores/workspaceblobstore/paths/datasets/got_persona_dataset_100.csv\"\ntransformations:\n  - read_delimited:\n      path_column: Path\n      include_path_column: false\n      encoding: utf8\n      support_multi_line: false\n      delimiter: \",\"\n      empty_as_string: false\n      partition_size: 20971520\n      header: all_files_same_headers\n      infer_column_types: false\n  - convert_column_types:\n      - columns: house_affiliation\n        column_type: string\n      - columns: ruthlessness_1to5\n        column_type: int\n      - columns: trait_impulsive\n        column_type: boolean\n      - columns: trait_vengeful\n        column_type: boolean\n      - columns: trait_loyal\n        column_type: boolean\n      - columns: combat_skill_1to5\n        column_type: int\n      - columns

In [14]:
from azure.ai.ml import MLClient, Input, Output, command
from azure.ai.ml.entities import Environment
from azure.identity import DefaultAzureCredential
from azure.ai.ml.dsl import pipeline
from azureml.core import Workspace, Dataset

credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential=credential)
ws = Workspace.from_config()

print(f"✓ Connected to: {ml_client.workspace_name}")

Found the config file in: /config.json


✓ Connected to: demeyer-robin-ml


In [15]:
# Load your existing tabular dataset
dataset = Dataset.get_by_name(ws, name='got-persona-dataset', version='1')
df = dataset.to_pandas_dataframe()

# Save as data.csv (required by your prepare_component.py)
df.to_csv('data.csv', index=False)
print(f"✓ Dataset shape: {df.shape}")

# Upload to datastore
datastore = ws.get_default_datastore()
datastore.upload_files(
    files=['data.csv'],
    target_path='got_input/',
    overwrite=True
)
print("✓ Uploaded to: got_input/data.csv")

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
✓ Dataset shape: (100, 21)
Uploading an estimated of 1 files
Uploading data.csv
Uploaded data.csv, 1 files out of an estimated total of 1
Uploaded 1 files
✓ Uploaded to: got_input/data.csv


In [16]:
env = Environment(
    name="got-prepare-env",
    conda_file="conda.yaml",  # Uses your existing file
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)
env = ml_client.environments.create_or_update(env)
print(f"✓ Environment: {env.name}")

✓ Environment: got-prepare-env


In [25]:
from azure.ai.ml import load_component

# First, create a component YAML file
component_yaml = """
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: prepare_got_data
display_name: Prepare GOT Data
version: 1
type: command

inputs:
  input_folder:
    type: uri_folder
  target_col:
    type: string
    default: house_affiliation
  test_size:
    type: number
    default: 0.2
  seed:
    type: integer
    default: 42
  stratify:
    type: integer
    default: 1

outputs:
  out_train:
    type: uri_folder
  out_test:
    type: uri_folder

code: .

environment: azureml:got-prepare-env@latest

command: >
  python prepare_component.py
  --input_folder ${{inputs.input_folder}}
  --target_col ${{inputs.target_col}}
  --test_size ${{inputs.test_size}}
  --seed ${{inputs.seed}}
  --stratify ${{inputs.stratify}}
  --out_train ${{outputs.out_train}}
  --out_test ${{outputs.out_test}}
"""

# Save YAML
with open('prepare_component.yaml', 'w') as f:
    f.write(component_yaml)

# Load and register the component
prepare_component = load_component(source='prepare_component.yaml')
prepare_component = ml_client.components.create_or_update(prepare_component)

print(f"✓ Component registered!")
print(f"  Name: {prepare_component.name}")
print(f"  Version: {prepare_component.version}")

Uploading Exam (0.07 MBs): 100%|██████████| 73460/73460 [00:00<00:00, 573894.40it/s]




✓ Component registered!
  Name: prepare_got_data
  Version: 1


In [26]:
@pipeline(name="got_data_preparation")
def data_prep_pipeline(input_data, target_col="house_affiliation"):
    step = prepare_component(
        input_folder=input_data,
        target_col=target_col,
        test_size=0.2,
        seed=42,
        stratify=1
    )
    return {
        "train_data": step.outputs.out_train,
        "test_data": step.outputs.out_test,
    }

print("✓ Pipeline defined")

✓ Pipeline defined


In [27]:
pipeline_job = data_prep_pipeline(
    input_data=Input(
        type="uri_folder",
        path="azureml://datastores/workspaceblobstore/paths/got_input/"
    ),
    target_col="house_affiliation"
)

# Set default compute for the pipeline
pipeline_job.settings.default_compute = "RDM-compute"

pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job,
    experiment_name="got_data_preparation"
)

print(f"\n✓ Pipeline submitted!")
print(f"View at: {pipeline_job.studio_url}")

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored



✓ Pipeline submitted!
View at: https://ml.azure.com/runs/stoic_candle_tyc55kplsm?wsid=/subscriptions/ebc57341-e881-4473-9659-d8076026bd45/resourcegroups/azure-ai/workspaces/demeyer-robin-ml&tid=4ded4bb1-6bff-42b3-aed7-6a36a503bf7a


In [38]:
training_env = Environment(
    name="got-train-dt-env",
    conda_file="conda_train.yaml",  # Uses your existing file
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)
training_env = ml_client.environments.create_or_update(training_env)
print(f"✓ Training environment registered: {training_env.name}")

ActivityCompleted: Activity=Environment.CreateOrUpdate, HowEnded=Failure, Duration=289.13 [ms], Exception=ResourceExistsError, ErrorCategory=UserError, ErrorMessage=(UserError) Environment got-train-dt-env with version 1 is already registered and cannot be changed.
Code: UserError
Message: Environment got-train-dt-env with version 1 is already registered and cannot be changed.


✓ Training environment registered: got-train-dt-env


In [46]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Create a registry client to access azureml registry components
registry_client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id=ml_client.subscription_id,
    resource_group_name=ml_client.resource_group_name,
    registry_name="azureml"
)

# Get the register_model component from the registry
register_model_component = registry_client.components.get(
    name="register_model",
    version="0.0.21"
)

print("✓ Loaded register_model component from azureml registry")
print(f"  Component: {register_model_component.name}")
print(f"  Version: {register_model_component.version}")

✓ Loaded register_model component from azureml registry
  Component: register_model
  Version: 0.0.21


In [47]:
train_component_code_no_registration = """import argparse
import json
import os
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib
import mlflow
import mlflow.sklearn


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--train_ready", type=str, required=True)
    parser.add_argument("--test_ready", type=str, required=True)
    parser.add_argument("--target_col", type=str, default="house_affiliation")
    parser.add_argument("--max_depth", type=int, default=10)
    parser.add_argument("--min_samples_split", type=int, default=2)
    parser.add_argument("--min_samples_leaf", type=int, default=1)
    parser.add_argument("--random_state", type=int, default=42)
    parser.add_argument("--model_output", type=str, required=True)
    args = parser.parse_args()

    print("="*60)
    print("Decision Tree Training Component")
    print("="*60)

    print("\\n1. Loading data...")
    X_train_path = os.path.join(args.train_ready, "X_train.csv")
    y_train_path = os.path.join(args.train_ready, "y_train.csv")
    X_test_path = os.path.join(args.test_ready, "X_test.csv")
    y_test_path = os.path.join(args.test_ready, "y_test.csv")

    for p in [X_train_path, y_train_path, X_test_path, y_test_path]:
        if not os.path.exists(p):
            raise FileNotFoundError(f"Missing required file: {p}")

    X_train = pd.read_csv(X_train_path)
    y_train = pd.read_csv(y_train_path)[args.target_col].astype(str)
    X_test = pd.read_csv(X_test_path)
    y_test = pd.read_csv(y_test_path)[args.target_col].astype(str)

    print(f"   Train: {X_train.shape[0]} samples, {X_train.shape[1]} features")
    print(f"   Test: {X_test.shape[0]} samples")

    mlflow.start_run()

    print("\\n2. Training Decision Tree Classifier...")
    mlflow.log_param("max_depth", args.max_depth)
    mlflow.log_param("min_samples_split", args.min_samples_split)
    mlflow.log_param("min_samples_leaf", args.min_samples_leaf)
    mlflow.log_param("random_state", args.random_state)
    mlflow.log_param("target_col", args.target_col)

    clf = DecisionTreeClassifier(
        max_depth=args.max_depth,
        min_samples_split=args.min_samples_split,
        min_samples_leaf=args.min_samples_leaf,
        random_state=args.random_state,
    )
    clf.fit(X_train, y_train)
    print("   ✓ Model trained")

    print("\\n3. Evaluating model...")
    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)
    
    train_acc = float(accuracy_score(y_train, y_pred_train))
    test_acc = float(accuracy_score(y_test, y_pred_test))
    
    print(f"   Training accuracy: {train_acc:.4f}")
    print(f"   Test accuracy: {test_acc:.4f}")

    mlflow.log_metric("train_accuracy", train_acc)
    mlflow.log_metric("test_accuracy", test_acc)

    report = classification_report(y_test, y_pred_test, output_dict=True, zero_division=0)
    
    for label, metrics in report.items():
        if isinstance(metrics, dict):
            mlflow.log_metric(f"{label}_precision", metrics['precision'])
            mlflow.log_metric(f"{label}_recall", metrics['recall'])
            mlflow.log_metric(f"{label}_f1", metrics['f1-score'])

    print("\\n4. Saving model...")
    os.makedirs(args.model_output, exist_ok=True)
    model_path = os.path.join(args.model_output, "model.pkl")
    joblib.dump(clf, model_path)
    print(f"   Model saved: {model_path}")

    # Log model to MLflow (but don't register yet - that's done in separate step)
    mlflow.sklearn.log_model(clf, "decision_tree_model")

    report_path = os.path.join(args.model_output, "metrics.json")
    with open(report_path, 'w') as f:
        json.dump({
            "train_accuracy": train_acc,
            "test_accuracy": test_acc,
            "classification_report": report
        }, f, indent=2)

    mlflow.end_run()

    print("\\n" + "="*60)
    print("✓ Training completed successfully!")
    print("="*60)


if __name__ == "__main__":
    main()
"""

with open('train_component.py', 'w') as f:
    f.write(train_component_code_no_registration)

print("✓ train_component.py updated (without registration)")

✓ train_component.py updated (without registration)


In [48]:
train_component_yaml = """
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_decision_tree
display_name: Train Decision Tree Classifier
version: 3
type: command

inputs:
  train_ready:
    type: uri_folder
  test_ready:
    type: uri_folder
  target_col:
    type: string
    default: house_affiliation
  max_depth:
    type: integer
    default: 10
  min_samples_split:
    type: integer
    default: 2
  min_samples_leaf:
    type: integer
    default: 1
  random_state:
    type: integer
    default: 42

outputs:
  model_output:
    type: uri_folder

code: .
environment: azureml:got-train-dt-env@latest

command: >
  python train_component.py
  --train_ready ${{inputs.train_ready}}
  --test_ready ${{inputs.test_ready}}
  --target_col ${{inputs.target_col}}
  --max_depth ${{inputs.max_depth}}
  --min_samples_split ${{inputs.min_samples_split}}
  --min_samples_leaf ${{inputs.min_samples_leaf}}
  --random_state ${{inputs.random_state}}
  --model_output ${{outputs.model_output}}
"""

with open('train_component.yaml', 'w') as f:
    f.write(train_component_yaml)

from azure.ai.ml import load_component
train_component = load_component(source='train_component.yaml')
train_component = ml_client.components.create_or_update(train_component)

print(f"✓ Training component v3 registered")

Uploading Exam (0.12 MBs): 100%|██████████| 115476/115476 [00:00<00:00, 1533818.22it/s]




✓ Training component v3 registered


In [51]:
@pipeline(
    name="got_complete_pipeline_with_registration",
    description="Complete pipeline: Data prep + Training + Model registration"
)
def complete_got_pipeline_3steps(input_data, target_col="house_affiliation", model_name="got_decision_tree_model"):
    # Step 1: Data preparation
    prep_step = prepare_component(
        input_folder=input_data,
        target_col=target_col,
        test_size=0.2,
        seed=42,
        stratify=1
    )
    
    # Step 2: Training
    train_step = train_component(
        train_ready=prep_step.outputs.out_train,
        test_ready=prep_step.outputs.out_test,
        target_col=target_col,
        max_depth=10,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=42
    )
    
    # Step 3: Register model using Azure's component
    register_step = register_model_component(
        model_path=train_step.outputs.model_output,
        model_name=model_name,
        model_type="custom_model"
    )
    
    return {
        "train_data": prep_step.outputs.out_train,
        "test_data": prep_step.outputs.out_test,
        "trained_model": train_step.outputs.model_output,
        "registered_model": register_step.outputs.registration_details_folder  # ← Fixed!
    }

print("✓ Complete 3-step pipeline defined")

✓ Complete 3-step pipeline defined


In [53]:
# Use Azure's pre-built curated environment (no build needed!)
from azure.ai.ml.entities import Environment

curated_env_name = "AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:1"

# Update training component to use curated environment
train_component_yaml_curated = """
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_decision_tree
display_name: Train Decision Tree Classifier
version: 4
type: command

inputs:
  train_ready:
    type: uri_folder
  test_ready:
    type: uri_folder
  target_col:
    type: string
    default: house_affiliation
  max_depth:
    type: integer
    default: 10
  min_samples_split:
    type: integer
    default: 2
  min_samples_leaf:
    type: integer
    default: 1
  random_state:
    type: integer
    default: 42

outputs:
  model_output:
    type: uri_folder

code: .
environment: azureml:AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:1

command: >
  python train_component.py
  --train_ready ${{inputs.train_ready}}
  --test_ready ${{inputs.test_ready}}
  --target_col ${{inputs.target_col}}
  --max_depth ${{inputs.max_depth}}
  --min_samples_split ${{inputs.min_samples_split}}
  --min_samples_leaf ${{inputs.min_samples_leaf}}
  --random_state ${{inputs.random_state}}
  --model_output ${{outputs.model_output}}
"""

with open('train_component.yaml', 'w') as f:
    f.write(train_component_yaml_curated)

from azure.ai.ml import load_component
train_component = load_component(source='train_component.yaml')
train_component = ml_client.components.create_or_update(train_component)

print(f"✓ Training component v4 registered with curated environment")
print("  This environment is pre-built and won't need to build!")

Uploading Exam (0.13 MBs): 100%|██████████| 134605/134605 [00:00<00:00, 1735704.35it/s]




✓ Training component v4 registered with curated environment
  This environment is pre-built and won't need to build!


In [54]:
complete_job = complete_got_pipeline_3steps(
    input_data=Input(
        type="uri_folder",
        path="azureml://datastores/workspaceblobstore/paths/got_input/"
    ),
    target_col="house_affiliation",
    model_name="got_decision_tree_model"
)

complete_job.settings.default_compute = "serverless"

complete_job = ml_client.jobs.create_or_update(
    complete_job,
    experiment_name="got_complete_pipeline"
)

print(f"\n✓ Complete 3-step pipeline submitted!")
print(f"View at: {complete_job.studio_url}")

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored



✓ Complete 3-step pipeline submitted!
View at: https://ml.azure.com/runs/maroon_cup_7fxbbtyhyb?wsid=/subscriptions/ebc57341-e881-4473-9659-d8076026bd45/resourcegroups/azure-ai/workspaces/demeyer-robin-ml&tid=4ded4bb1-6bff-42b3-aed7-6a36a503bf7a


In [55]:
# First, get the latest training component
train_component_latest = ml_client.components.get(
    name="train_decision_tree",
    label="latest"  # This gets the absolute latest version
)

print(f"Latest component version: {train_component_latest.version}")

Latest component version: 4


In [56]:
train_component_yaml_v5 = """
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_decision_tree
display_name: Train Decision Tree Classifier
version: 5
type: command

inputs:
  train_ready:
    type: uri_folder
  test_ready:
    type: uri_folder
  target_col:
    type: string
    default: house_affiliation
  max_depth:
    type: integer
    default: 10
  min_samples_split:
    type: integer
    default: 2
  min_samples_leaf:
    type: integer
    default: 1
  random_state:
    type: integer
    default: 42

outputs:
  model_output:
    type: uri_folder

code: .
environment: azureml:AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:1

command: >
  python train_component.py
  --train_ready ${{inputs.train_ready}}
  --test_ready ${{inputs.test_ready}}
  --target_col ${{inputs.target_col}}
  --max_depth ${{inputs.max_depth}}
  --min_samples_split ${{inputs.min_samples_split}}
  --min_samples_leaf ${{inputs.min_samples_leaf}}
  --random_state ${{inputs.random_state}}
  --model_output ${{outputs.model_output}}
"""

with open('train_component.yaml', 'w') as f:
    f.write(train_component_yaml_v5)

print("✓ train_component.yaml v5 created with curated environment")

✓ train_component.yaml v5 created with curated environment


In [57]:
from azure.ai.ml import load_component

train_component_v5 = load_component(source='train_component.yaml')
train_component_v5 = ml_client.components.create_or_update(train_component_v5)

print(f"✓ Component v5 registered!")
print(f"  Name: {train_component_v5.name}")
print(f"  Version: {train_component_v5.version}")
print(f"  Environment: AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:1 (CURATED)")

Uploading Exam (0.13 MBs): 100%|██████████| 128462/128462 [00:00<00:00, 1440977.00it/s]




✓ Component v5 registered!
  Name: train_decision_tree
  Version: 5
  Environment: AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:1 (CURATED)


In [58]:
@pipeline(
    name="got_complete_pipeline_with_registration",
    description="Complete pipeline: Data prep + Training + Model registration"
)
def complete_got_pipeline_v5(input_data, target_col="house_affiliation", model_name="got_decision_tree_model"):
    # Step 1: Data preparation
    prep_step = prepare_component(
        input_folder=input_data,
        target_col=target_col,
        test_size=0.2,
        seed=42,
        stratify=1
    )
    
    # Step 2: Training with V5 (curated environment)
    train_step = train_component_v5(
        train_ready=prep_step.outputs.out_train,
        test_ready=prep_step.outputs.out_test,
        target_col=target_col,
        max_depth=10,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=42
    )
    
    # Step 3: Register model
    register_step = register_model_component(
        model_path=train_step.outputs.model_output,
        model_name=model_name,
        model_type="custom_model"
    )
    
    return {
        "train_data": prep_step.outputs.out_train,
        "test_data": prep_step.outputs.out_test,
        "trained_model": train_step.outputs.model_output,
        "registered_model": register_step.outputs.registration_details_folder
    }

print("✓ Pipeline defined with component v5")

✓ Pipeline defined with component v5


In [59]:
complete_job_v5 = complete_got_pipeline_v5(
    input_data=Input(
        type="uri_folder",
        path="azureml://datastores/workspaceblobstore/paths/got_input/"
    ),
    target_col="house_affiliation",
    model_name="got_decision_tree_model"
)

complete_job_v5.settings.default_compute = "serverless"

complete_job_v5 = ml_client.jobs.create_or_update(
    complete_job_v5,
    experiment_name="got_complete_pipeline"
)

print(f"\n✓ Pipeline submitted with V5 (CURATED ENVIRONMENT)!")
print(f"View at: {complete_job_v5.studio_url}")
print("\nThis should start immediately - no building needed! 🚀")

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored



✓ Pipeline submitted with V5 (CURATED ENVIRONMENT)!
View at: https://ml.azure.com/runs/orange_quince_8bwp8spt2w?wsid=/subscriptions/ebc57341-e881-4473-9659-d8076026bd45/resourcegroups/azure-ai/workspaces/demeyer-robin-ml&tid=4ded4bb1-6bff-42b3-aed7-6a36a503bf7a

This should start immediately - no building needed! 🚀


In [62]:
print("Available compute targets:")
compute_list = []
for compute in ml_client.compute.list():
    print(f"  {compute.name} - {compute.type}")
    compute_list.append(compute.name)

if compute_list:
    print(f"\nYou can use: {compute_list[0]}")
else:
    print("\nOnly serverless available")

Available compute targets:
  RDM-compute - computeinstance

You can use: RDM-compute


In [63]:
# Cancel the stuck job
try:
    ml_client.jobs.cancel(complete_job_v5.name)
    print("✓ Cancelled stuck job")
except:
    print("Job already finished or cancelled")

# Re-run with explicit compute
complete_job_v5_retry = complete_got_pipeline_v5(
    input_data=Input(
        type="uri_folder",
        path="azureml://datastores/workspaceblobstore/paths/got_input/"
    ),
    target_col="house_affiliation",
    model_name="got_decision_tree_model"
)

# Use first available compute or serverless
if compute_list:
    complete_job_v5_retry.settings.default_compute = compute_list[0]
    print(f"Using compute: {compute_list[0]}")
else:
    complete_job_v5_retry.settings.default_compute = "serverless"
    print("Using serverless compute")

complete_job_v5_retry = ml_client.jobs.create_or_update(
    complete_job_v5_retry,
    experiment_name="got_complete_pipeline"
)

print(f"\n✓ Pipeline re-submitted!")
print(f"View at: {complete_job_v5_retry.studio_url}")
print("\nWatch it closely - training should start within 2-3 minutes with curated env")

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored



✓ Pipeline re-submitted!
View at: https://ml.azure.com/runs/zen_planet_clwdz5zp5g?wsid=/subscriptions/ebc57341-e881-4473-9659-d8076026bd45/resourcegroups/azure-ai/workspaces/demeyer-robin-ml&tid=4ded4bb1-6bff-42b3-aed7-6a36a503bf7a

Watch it closely - training should start within 2-3 minutes with curated env
